# CAS AML project - Module 1 & 2

In [6]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import os
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path

# To make this notebook's output stable across runs
np.random.seed(42)

# Paths
DATADIR = os. getcwd() + '/../data'

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Data preparation

### Target variable: precipitation time series

**Dataset**: RhiresD, which is a gridded daily precipitation dataset over Switzerland provided by MeteoSwiss. It is based on a spatial interpolation of rain-gauge data. The grid resolution is 1 km, but the effective resolution is in the order of 15-20 km.


**Aggregations levels**: The gridded dataset has been averaged over different regions:
* 12 climatic regions
* 5 aggregated regions
* the whole country

In [8]:
# Read precipitation file and get events over threshold
precip = pd.read_csv(DATADIR + '/MeteoSwiss/precip_regions.csv')
precip_xtreme = precip.copy()

for key, ts in precip_xtreme.iteritems():
    if key in ['year', 'month', 'day']: continue
    precip_xtreme[key] = ts > ts.quantile(0.95)

### Predictors: meteorological fields

**Dataset**: ERA5

## Unsupervised learning approaches

### PCA

### K-means clustering

## Supervised learning approaches

### Linear regression for precipitation values

In [11]:
# Open geopotential data
z = xr.open_mfdataset(DATADIR + '/ERA5/geopotential/*.nc', combine='by_coords')

# Convert to hPa
z.z.values = z.z.values/9.80665
lon = z.lon
lat = z.lat

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
http://xarray.pydata.org/en/stable/user-guide/io.html 
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html

### Logistic regression for extreme events

### Random forest

## Deep learning approaches